In [2]:
!pip install azure-datalake-store 

  Using cached azure_datalake_store-0.0.48-py2.py3-none-any.whl (53 kB)
  Using cached adal-1.2.2-py2.py3-none-any.whl (53 kB)


In [1]:
from azure.datalake.store import core, lib, multithread


In [2]:
from pyspark.sql import SparkSession


In [3]:
spark = SparkSession.builder \
            .master("local[8]") \
            .appName("airflow_app") \
            .config('spark.executor.memory', '16g') \
            .config('spark.driver.memory', '16g') \
            .config('spark.sql.execution.pandas.respectSessionTimeZone', False) \
            .config("spark.driver.maxResultSize", "2048MB") \
            .config("spark.port.maxRetries", "100") \
            .config("spark.sql.execution.arrow.enabled", "true") \
            .getOrCreate()


In [4]:
adlCreds = lib.auth(url_suffix='raizenprd01', resource='https://datalake.azure.net/')

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B3B4PQPLF to authenticate.


In [5]:
spark.conf.set("fs.adl.oauth2.access.token.provider.type", "RefreshToken")
spark.conf.set("fs.adl.oauth2.client.id", adlCreds.token['client'])
spark.conf.set("fs.adl.oauth2.refresh.token", adlCreds.token['refreshToken'])


In [6]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import StringType,StructField, StructType  

In [7]:
df_vendas_dia = spark.read.parquet('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/03_primary/vendas_dia').where((F.col('Date') < F.lit('2019-08-01'))&(F.col('Date') > F.lit('2019-07-01')))

In [14]:
schema_mes = StructType([
        StructField('_c0', StringType(), True),
        StructField('Empr', StringType(), True),
        StructField('Cent', StringType(), True),
        StructField('Nota Fiscal', StringType(), True),
        StructField('Data Saida', StringType(), True),
        StructField('Data Criacao', StringType(), True),
        StructField('UF', StringType(), True),
        StructField('Faturamento', StringType(), True),
        StructField('Itm', StringType(), True),
        StructField('Vencimento', StringType(), True),
        StructField('TpOV', StringType(), True),
        StructField('Ordem Vendas', StringType(), True),
        StructField('IBM', StringType(), True),
        StructField('Razao Social', StringType(), True),
        StructField('Canal', StringType(), True),
        StructField('Material', StringType(), True),
        StructField('Descricao', StringType(), True),
        StructField(' Quantidade', StringType(), True),
        StructField('UM', StringType(), True),
        StructField('Cond.Pgto', StringType(), True),
        StructField('Inco', StringType(), True),
        StructField('C.Exp', StringType(), True),
        StructField('Valor item', StringType(), True),
        StructField('Valor ZPFL', StringType(), True),
        StructField('Valor ZPFN', StringType(), True),
        StructField('Valor ZENC', StringType(), True),
        StructField('Imposto', StringType(), True),
        StructField('Valor ZDB1', StringType(), True),
        StructField('Target Block', StringType(), True),
        StructField('UN', StringType(), True),
        StructField('Valor ZDDB', StringType(), True),
        StructField('Valor ZDIA', StringType(), True),
        StructField('Valor ZDFR', StringType(), True),
        StructField('Valor ICM3', StringType(), True),
        StructField('Valor ICS3', StringType(), True),
        StructField('Rep.Vendas', StringType(), True),
        StructField('Acesso ZPFL', StringType(), True),
        StructField('Volume', StringType(), True),
        StructField('UN_', StringType(), True),
        StructField('Hora Fat', StringType(), True),
        StructField('Referência', StringType(), True),
    ])

In [15]:
df_mes = spark.read.format('csv')\
        .option('sep','|').schema(schema_mes)\
        .load('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/01_raw/vendas_dia_complemento/mes')\
        .where("Empr is not null and Empr <> 'Empr' ").select('Data Saida','Cent','Material','IBM','Volume','Canal')

In [16]:
df_mes = df_mes\
    .withColumn("Date", F.to_date(F.col("Data Saida"), "dd.MM.yyyy"))\
    .withColumn("Volume",F.regexp_replace(F.regexp_replace(df_mes.Volume,"\.",""),",",".").cast("double"))\
    .withColumnRenamed('Canal','Segmento').withColumnRenamed('Cent','Base')\
    .withColumnRenamed('Material','SKU')\
    .where(F.col('Date') < F.lit('2019-10-01'))\
    .select('Date','Base','SKU','IBM','Volume','Segmento')

In [17]:
df_mes = df_mes.groupBy("Date","Base","SKU","IBM","Segmento").agg({"Volume": "sum"})

In [18]:
df_mes.show(5)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in r

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-12ee3fb1134d>", line 1, in <module>
    df_mes.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 378, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o242.showString

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-12ee3fb1134d>", line 1, in <module>
    df_mes.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 378, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o242.showString

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-12ee3fb1134d>", line 1, in <module>
    df_mes.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 378, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o242.showString

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-12ee3fb1134d>", line 1, in <module>
    df_mes.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 378, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o242.showString

During handling of the above exception, another ex

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-12ee3fb1134d>", line 1, in <module>
    df_mes.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 378, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o242.showString

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-12ee3fb1134d>", line 1, in <module>
    df_mes.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 378, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o242.showString

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-12ee3fb1134d>", line 1, in <module>
    df_mes.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 378, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o242.showString

During handling of the above exception, another ex

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39031)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-12ee3fb1134d>", line 1, in <module>
    df_mes.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 378, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o242.showString

During handling of the above exception, another ex

Py4JError: An error occurred while calling o242.showString

In [15]:
df_vendas_dia.show(5)

+-------------------+----+--------+-------+--------+--------+
|               Date|Base|     SKU|    IBM|  Volume|Segmento|
+-------------------+----+--------+-------+--------+--------+
|2019-07-10 00:00:00|ABAH|15521801|1032793| 26969.0|       A|
|2019-07-10 00:00:00|ABAH|15526801|1000030|142958.0|       A|
|2019-07-10 00:00:00|ABAR|15521801|1033788|   519.0|       A|
|2019-07-10 00:00:00|ABAR|15532801|1056551|   708.0|       A|
|2019-07-10 00:00:00|ABCH|15521801|1035814|     0.0|       A|
+-------------------+----+--------+-------+--------+--------+
only showing top 5 rows



In [15]:
df_vendas_dia.groupBy("Date", "Base", "SKU", "IBM","Segmento").agg({"Volume": "sum"}).show()

+-------------------+----+--------+-------+--------+-----------+
|               Date|Base|     SKU|    IBM|Segmento|sum(Volume)|
+-------------------+----+--------+-------+--------+-----------+
|2019-07-10 00:00:00|BBAR|22150801|1023648|       V|     5050.0|
|2019-07-10 00:00:00|BBAR|27007801|1031446|       V|    10000.0|
|2019-07-10 00:00:00|BBAU|24318801|1038031|       V|     5000.0|
|2019-07-10 00:00:00|BCAX|24144801|1047499|       V|     5000.0|
|2019-07-10 00:00:00|BCAX|27007801|1039624|       V|     5000.0|
|2019-07-10 00:00:00|BCEL|24144801| 681261|       V|     5000.0|
|2019-07-10 00:00:00|BEMT|24319801|1040382|       C|    34500.0|
|2019-07-10 00:00:00|BEST|22146801|1046885|       V|    20000.0|
|2019-07-10 00:00:00|BEST|22149801|1038121|       V|     5000.0|
|2019-07-10 00:00:00|BFOR|22149801| 875562|       V|    10000.0|
|2019-07-10 00:00:00|BGOI|24141801|1062223|       C|    10000.0|
|2019-07-10 00:00:00| BIP|22149801|1030062|       V|     5000.0|
|2019-07-10 00:00:00| BIP

In [16]:
!pip install pyarrow

  Using cached pyarrow-0.16.0-cp37-cp37m-manylinux2014_x86_64.whl (63.1 MB)


In [16]:
import plotly.express as px


In [14]:
vendas_dia_group = df_vendas_dia.groupBy("Date", "Base", "SKU", "IBM","Segmento").agg({"Volume": "sum"})


In [18]:
vendas_dia_group.show()

+-------------------+----+--------+-------+--------+-----------+
|               Date|Base|     SKU|    IBM|Segmento|sum(Volume)|
+-------------------+----+--------+-------+--------+-----------+
|2019-07-10 00:00:00|BBAR|22150801|1023648|       V|     5050.0|
|2019-07-10 00:00:00|BBAR|27007801|1031446|       V|    10000.0|
|2019-07-10 00:00:00|BBAU|24318801|1038031|       V|     5000.0|
|2019-07-10 00:00:00|BCAX|24144801|1047499|       V|     5000.0|
|2019-07-10 00:00:00|BCAX|27007801|1039624|       V|     5000.0|
|2019-07-10 00:00:00|BCEL|24144801| 681261|       V|     5000.0|
|2019-07-10 00:00:00|BEMT|24319801|1040382|       C|    34500.0|
|2019-07-10 00:00:00|BEST|22146801|1046885|       V|    20000.0|
|2019-07-10 00:00:00|BEST|22149801|1038121|       V|     5000.0|
|2019-07-10 00:00:00|BFOR|22149801| 875562|       V|    10000.0|
|2019-07-10 00:00:00|BGOI|24141801|1062223|       C|    10000.0|
|2019-07-10 00:00:00| BIP|22149801|1030062|       V|     5000.0|
|2019-07-10 00:00:00| BIP

In [15]:
v_dia_group_pd = vendas_dia_group.orderBy("Date").toPandas()

/opt/spark/python/pyspark/sql/dataframe.py:2102: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.8.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


Py4JJavaError: An error occurred while calling o169.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13 in stage 7.0 failed 1 times, most recent failure: Lost task 13.0 in stage 7.0 (TID 433, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 451499 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:1877)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:1876)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scal

In [ ]:
px.line(v_dia_group_pd, x="Date", y=["meio", "sum(Volume)"])